In [ ]:
import os
import re
import random
import math
from math import pi
from scipy.integrate import quad
import numpy as np

ground_device = {}
device_list = ['CamID10066', 'CamID10870', 'CamID1093', 'CamID11160', 'CamID11331', 'CamID162', 
               'CamID17244', 'CamID19306', 'CamID19388', 'CamID204', 'CamID260', 'CamID3297', 'CamID3396', 
               'CamID3837', 'CamID3888', 'CamID4181', 'CamID4232', 'CamID4584', 'CamID4679', 'CamID4795', 
               'CamID4801', 'CamID5020', 'CamID5021', 'CamID623', 'CamID65', 'CamID6798', 'CamID684', 'CamID7211', 
               'CamID7233', 'CamID7371', 'CamID75', 'CamID8438', 'CamID858', 'CamID861', 'CamID8733', 'CamID8953', 
               'CamID9112', 'CamID9291', 'CamID9483', 'CamID9708', 'CamID9730']

for i in device_list:
    key = f'{i}'
    ground_device[key] = {'connections': {}}

# print(ground_device)
def read_device_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

        current_link = None
        link_times = []
        link_ranges = []
        skip_lines = 3  

        for line in lines[skip_lines:]:
            line = line.strip()
            if 'to' in line and re.match(r'CamID\d+ to LEOS\d{3}', line):
                if current_link:
                    cam_id = current_link.split(' to ')[0]
                    target_leos = current_link.split(' to ')[1]
#                     print(leos_id, target_leos)
                    ground_device[cam_id]['connections'][target_leos] = {
                        'Time': link_times,
                        'Range': link_ranges
                    }
                current_link = line
                link_times = []
                link_ranges = []
            elif line and re.match(r'\d{2} \w{3} \d{4} \d{2}:\d{2}:\d{2}', line):
                parts = re.split(r'\s{2,}', line)
                if len(parts) == 2:
                    timestamp = parts[0]
                    try:
                        distance = float(parts[1])
                        link_times.append(timestamp)
                        link_ranges.append(distance)
                    except ValueError:
                        continue

        if current_link:
            cam_id = current_link.split(' to ')[0]
            target_leos = current_link.split(' to ')[1]
            ground_device[cam_id]['connections'][target_leos] = {
                'Time': link_times,
                'Range': link_ranges
            }

file_path = r'D:\文件\SATCOM\log\SATDATA\Walker-Star\Ground-Space\CamID-Huge.txt'
read_device_data(file_path)

# physical params

random.seed(2024)


fc = 193 * 10**12 #载波频率 Hz
c = 3 * 10**8 #光速 m/s
lamb = c / fc #波长 m
print(f"Wavelength (m): {lamb}")
Bandwidth = 0.02 * fc #带宽 Hz
# Bandwidth = 0.2 * 10**9 #通信带宽
eta = 0.8
U = 25
SNRth = 10**(-110/10)
sigma_p = 0.05  # 根据你的需求调整
theta = 0.01  # 对准误差角 (弧度)
theta_3dB = 0.1  # 3dB波束宽度 (弧度)
G0 = 4 * math.log(2) / (theta_3dB ** 2)
# 计算对准损失
def pointing_loss(theta, theta_3dB):
    L_P = math.exp(-G0 * (theta ** 2))
    return L_P

def pointing_loss_pdf(theta, G0, sigma_p):
    return ( sigma_p * np.sqrt(G0) *theta ** (1 / (2 * G0 * sigma_p ** 2) - 1)) / (np.sqrt(- np.pi * np.log(theta)))


min_power_w = 31.6 / 1000 
# min_power_w = 5
max_power_w = 5
gain = 81.67
min_gain = 10 ** (gain / 10)
max_gain = 10 ** (gain / 10)

transmit_gain = [random.uniform(max_gain, max_gain) for _ in range(240)]
receiver_gain = [random.uniform(min_gain, min_gain) for _ in range(240)]
# effs = [random.choice([0.6, 0.7, 0.8]) for _ in range(80)]
effs = [random.choice([0.8]) for _ in range(240)]
power_values_watts = [random.uniform(max_power_w, max_power_w) for _ in range(240)]

k = 1.38e-23  # Boltzmann constant in J/K
T_b = 6000  # Solar brightness temperature in K
T_0 = 1000  # Typical system noise temperature in K (assumed value, can be changed)
T_CMB = 2.725  # Cosmic Microwave Background temperature in K

# Total noise temperature
T_total = T_b + T_0 + T_CMB

# Noise power calculation
P_N = k * T_total * Bandwidth
print("PN",P_N)
s = 242 * 1e6 * 8

# Free space path loss calculation
def PL(lamb, distance):
    L_FS_dB = 20 * math.log10(distance) + 20 * math.log10(fc/(10**6)) + 32.44
#     print(L_FS_dB)
    passloss = 10**(L_FS_dB/10)
    return passloss

# Receiver power calculation
def receiver_power(transmit_power, eff, trans_gain, recei_gain, path_loss, beam_loss):
    power = transmit_power * eff * trans_gain * recei_gain / path_loss * beam_loss
    return power

# Achievable rate calculation
def rate(Bandwidth, Receiver, Noise):
#     Receiver_W = 
    SNR = Receiver / Noise
    achievable_rate = Bandwidth * math.log2(1 + SNR)
    return achievable_rate, SNR

# Energy weight calculation
def weight(rate, s, U, transmit_power):
    energy = s * transmit_power / (U * rate)
    time = s / (U * rate)
    return energy, time


satellite = {}


for j in range(1, 9):
    for i in range(1 + j * 100, 31 + j * 100):
        key = f'LEOS{i}'
        satellite[key] = {'cameras': [], 
                          'connections': {}, 
                          'efficiency': None,
                          'transmit_power': None,
                          'transmit_gain': None,
                          'receiver_gain': None
                         }

def read_satellite_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

        current_link = None
        link_times = []
        link_ranges = []
        skip_lines = 3

        min_distance_links = {}

        for line in lines[skip_lines:]:
            line = line.strip()
            if 'to' in line and re.match(r'LEOS\d{3} to LEOS\d{3}', line):
                if current_link:
                    leos_id = current_link.split(' to ')[0]
                    target_leos = current_link.split(' to ')[1]
                    leos_id_num = int(re.search(r'\d+', leos_id).group())
                    target_leos_num = int(re.search(r'\d+', target_leos).group())
                    leos_id_orbit = leos_id_num // 100
                    target_leos_orbit = target_leos_num // 100

                    if leos_id_orbit != target_leos_orbit:

                        min_distance_key = (leos_id, target_leos_orbit)
                        min_distance = min(link_ranges) if link_ranges else float('inf')
                        
                        if min_distance_key not in min_distance_links or min_distance < min_distance_links[min_distance_key][0]:
                            min_distance_links[min_distance_key] = (min_distance, link_times, link_ranges)
                    else:

                        if (abs(leos_id_num - target_leos_num) != 1 and abs(leos_id_num - target_leos_num) != 29):
                            current_link = line
                            link_times = []
                            link_ranges = []
                            continue
                        satellite[leos_id]['connections'][target_leos] = {
                            'Time': link_times,
                            'Range': link_ranges
                        }

                current_link = line
                link_times = []
                link_ranges = []
            elif line and re.match(r'\d{2} \w{3} \d{4} \d{2}:\d{2}:\d{2}', line):
                parts = re.split(r'\s{2,}', line)
                if len(parts) == 2:
                    timestamp = parts[0]
                    try:
                        distance = float(parts[1])
                        link_times.append(timestamp)
                        link_ranges.append(distance)
                    except ValueError:
                        continue

        if current_link:
            leos_id = current_link.split(' to ')[0]
            target_leos = current_link.split(' to ')[1]
            leos_id_num = int(re.search(r'\d+', leos_id).group())
            target_leos_num = int(re.search(r'\d+', target_leos).group())
            leos_id_orbit = leos_id_num // 100
            target_leos_orbit = target_leos_num // 100

            if leos_id_orbit != target_leos_orbit:
                min_distance_key = (leos_id, target_leos_orbit)
                min_distance = min(link_ranges) if link_ranges else float('inf')
                
                if min_distance_key not in min_distance_links or min_distance < min_distance_links[min_distance_key][0]:
                    min_distance_links[min_distance_key] = (min_distance, link_times, link_ranges)
            else:
                satellite[leos_id]['connections'][target_leos] = {
                    'Time': link_times,
                    'Range': link_ranges
                }

        for (leos_id, target_leos_orbit), (min_distance, link_times, link_ranges) in min_distance_links.items():
            target_leos = None
            for key in satellite.keys():
                if key.startswith(f'LEOS{target_leos_orbit}'):
                    target_leos = key
                    break
            if target_leos:
                satellite[leos_id]['connections'][target_leos] = {
                    'Time': link_times,
                    'Range': link_ranges
                }


file_path = r'D:\文件\SATCOM\log\SATDATA\Walker-Star\Inter-satellite\LEOS-Huge.txt'
read_satellite_data(file_path)

for idx, key in enumerate(satellite.keys()):
    satellite[key]['efficiency'] = effs[idx]
    satellite[key]['transmit_power'] = power_values_watts[idx]
    satellite[key]['transmit_gain'] = transmit_gain[idx]
    satellite[key]['receiver_gain'] = receiver_gain[idx]

Wavelength (m): 1.5544041450777201e-06
PN 3.7302115530000006e-07


In [2]:
from datetime import datetime, timedelta
from collections import Counter
import pickle

# 定义起始时间和结束时间
start_time = '25 Jun 2024 02:00:00.000'
stop_time = '25 Jun 2024 02:05:00.000'

# 将字符串转换为datetime对象
start_time_dt = datetime.strptime(start_time, '%d %b %Y %H:%M:%S.%f')
stop_time_dt = datetime.strptime(stop_time, '%d %b %Y %H:%M:%S.%f')

# 计算总时长和每个5分钟段的长度
total_duration = stop_time_dt - start_time_dt
segment_duration = total_duration / 1

# 生成时间段
time_segments = [start_time_dt + i * segment_duration for i in range(2)]

# 创建一个字典来存储所有时间段参数
time_params = {}

for i in range(len(time_segments) - 1):
    segment_start = time_segments[i]
    segment_end = time_segments[i + 1]
    
    # 前50秒为星地通信时延
    comm_start = segment_start
    comm_end = comm_start + timedelta(seconds=50)
    
    # 后250秒为星上time slot，进一步分为25个10秒的time frame
    slot_start = comm_end
    slot_end = segment_end
    
    # 生成25个time frame，每个10秒
    time_frames = []
    for j in range(25):
        frame_start = slot_start + timedelta(seconds=j * 10)
        frame_end = frame_start + timedelta(seconds=10)
        time_frames.append({
            'frame_start': frame_start.strftime('%d %b %Y %H:%M:%S.%f')[:-3],
            'frame_end': frame_end.strftime('%d %b %Y %H:%M:%S.%f')[:-3]
        })
    
    # 存储在字典中
    time_params[f'segment_{i + 1}'] = {
        'comm_start': comm_start.strftime('%d %b %Y %H:%M:%S.%f')[:-3],
        'comm_end': comm_end.strftime('%d %b %Y %H:%M:%S.%f')[:-3],
        'slot_start': slot_start.strftime('%d %b %Y %H:%M:%S.%f')[:-3],
        'slot_end': slot_end.strftime('%d %b %Y %H:%M:%S.%f')[:-3],
        'time_frames': time_frames
    }

# 创建名为 ground_device 的字典
ground_device = {}
device_list = ['CamID10066', 'CamID10870', 'CamID1093', 'CamID11160', 'CamID11331', 'CamID162', 
               'CamID17244', 'CamID19306', 'CamID19388', 'CamID204', 'CamID260', 'CamID3297', 'CamID3396', 
               'CamID3837', 'CamID3888', 'CamID4181', 'CamID4232', 'CamID4584', 'CamID4679', 'CamID4795', 
               'CamID4801', 'CamID5020', 'CamID5021', 'CamID623', 'CamID65', 'CamID6798', 'CamID684', 'CamID7211', 
               'CamID7233', 'CamID7371', 'CamID75', 'CamID8438', 'CamID858', 'CamID861', 'CamID8733', 'CamID8953', 
               'CamID9112', 'CamID9291', 'CamID9483', 'CamID9708', 'CamID9730']

for i in device_list:
    key = f'{i}'
    ground_device[key] = {'connections': {}}

# 读取设备数据
def read_device_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

        current_link = None
        link_times = []
        link_ranges = []
        skip_lines = 3  # 跳过文件开头的三行

        for line in lines[skip_lines:]:
            line = line.strip()
            if 'to' in line and re.match(r'CamID\d+ to LEOS\d{3}', line):
                if current_link:
                    cam_id = current_link.split(' to ')[0]
                    target_leos = current_link.split(' to ')[1]
                    ground_device[cam_id]['connections'][target_leos] = {
                        'Time': link_times,
                        'Range': link_ranges
                    }
                current_link = line
                link_times = []
                link_ranges = []
            elif line and re.match(r'\d{2} \w{3} \d{4} \d{2}:\d{2}:\d{2}', line):
                parts = re.split(r'\s{2,}', line)
                if len(parts) == 2:
                    timestamp = parts[0]
                    try:
                        distance = float(parts[1])
                        link_times.append(timestamp)
                        link_ranges.append(distance)
                    except ValueError:
                        continue

        # 最后一个链路检查
        if current_link:
            cam_id = current_link.split(' to ')[0]
            target_leos = current_link.split(' to ')[1]
            ground_device[cam_id]['connections'][target_leos] = {
                'Time': link_times,
                'Range': link_ranges
            }

# 使用函数读取设备数据
file_path = r'D:\文件\SATCOM\log\SATDATA\Walker-Star\Ground-Space\CamID.txt'
read_device_data(file_path)

# 初始化terminal
terminal = [set() for i in range(1)]
rootnode = [[] for i in range(1)]

index = 0
for segment, params in time_params.items():
    comm_start_str = params['comm_start']
    comm_start_dt = datetime.strptime(comm_start_str, '%d %b %Y %H:%M:%S.%f')
    closest_leos = {}
    
    for device, data in ground_device.items():
        min_distance = float('inf')
        closest_leos_id = None
        
        for leos_id, connections in data['connections'].items():
            for time_str, distance in zip(connections['Time'], connections['Range']):
                time_dt = datetime.strptime(time_str, '%d %b %Y %H:%M:%S.%f')
                time_diff = abs((time_dt - comm_start_dt).total_seconds())
                if time_diff < 10:  # 50秒内的记录
                    if distance < min_distance:
                        min_distance = distance
                        closest_leos_id = leos_id
                        
        if closest_leos_id:
            terminal[index].add(closest_leos_id)
            rootnode[index].append(closest_leos_id)
            closest_leos[device] = {
                'leos_id': closest_leos_id,
                'distance': min_distance
            }
    index += 1

# 将每个terminal的set转换为list
terminal = [sorted(list(t), key=lambda x: int(re.search(r'\d+', x).group())) for t in terminal]
print(terminal)
root_list = [Counter(i).most_common(1)[0][0] for i in rootnode]


[['LEOS108', 'LEOS109', 'LEOS208', 'LEOS209', 'LEOS210', 'LEOS309', 'LEOS312', 'LEOS313', 'LEOS404', 'LEOS407', 'LEOS408']]


In [3]:
print(root_list)

['LEOS408']


In [4]:
# 创建有向图G
G = {}

# 构建子图G0-G24
for j in range(1):
    time_frame_key = f'Frame_{j}'
    G[time_frame_key] = {}
    for i in range(25):
        subgraph_key = f'G{i}'
        G[time_frame_key][subgraph_key] = {}
        time_start_str = time_params[f'segment_{j + 1}']['time_frames'][i]['frame_start']
        time_start_dt = datetime.strptime(time_start_str, '%d %b %Y %H:%M:%S.%f')
        for sat1 in satellite:
            for sat2, connection in satellite[sat1]['connections'].items():
                if sat2 in satellite:
                    # 找到 time_start 在 connection['Time'] 中的索引
                    for idx, time_str in enumerate(connection['Time']):
                        time_dt = datetime.strptime(time_str, '%d %b %Y %H:%M:%S.%f')
                        timediff = (time_start_dt - time_dt).total_seconds()
                        if time_dt == time_start_dt or (0 < timediff < 10):
                            distance = connection['Range'][idx]
                            path_loss = PL(fc, distance)
#                             print("PL:", path_loss)
                            eff = satellite[sat2]['efficiency']
                            trans_power = satellite[sat1]['transmit_power']
                            trans_gain = satellite[sat1]['transmit_gain']
                            recei_gain = satellite[sat2]['receiver_gain']
                
#                             # Check if sat1 and sat2 are in the same orbit
                            sat1_orbit = int(re.search(r'(\d+)', sat1).group()) // 100
                            sat2_orbit = int(re.search(r'(\d+)', sat2).group()) // 100
                            if sat1_orbit == sat2_orbit:
                                L_P = 1 # Same orbit
                            else:
                                L_P = pointing_loss(theta, theta_3dB)  # Calculate pointing loss

                            recei_power = receiver_power(trans_power, eff, trans_gain, recei_gain, path_loss, L_P)
                            achievable_rate, SNR = rate(Bandwidth, recei_power, P_N)
                            gamma = SNRth / (recei_power / L_P)
#                 print(gamma)
                            outage, _ = quad(pointing_loss_pdf, 0, gamma, args=(G0, sigma_p))
#                             print(outage)
#                             if math.isnan(outage):
#                                 break
#                             else: 
                            energy, time = weight(achievable_rate, s, U, trans_power)
#                 print(f"Receiver Power (dB): {recei_power}")

                            G[time_frame_key][subgraph_key][(sat1, sat2)] = {
                                    'energy': energy,
                                    'rate': achievable_rate/(10**9),
                                    'outage': outage
                                    }
                            break  # 退出内部for循环，因为我们已经找到了匹配的时间
        print(f'Frame: {time_frame_key}, Subgraph {subgraph_key}, construction finished.')

# # # 打印部分结果以验证
# # for subgraph_key in list(G.keys())[:5]:  # 仅打印前5个子图
# #     print(f'Subgraph {subgraph_key}:')
# #     for link, details in G[subgraph_key].items():
# #         print(f'{link}: {details}')
# #     print()

Frame: Frame_0, Subgraph G0, construction finished.
Frame: Frame_0, Subgraph G1, construction finished.
Frame: Frame_0, Subgraph G2, construction finished.
Frame: Frame_0, Subgraph G3, construction finished.
Frame: Frame_0, Subgraph G4, construction finished.
Frame: Frame_0, Subgraph G5, construction finished.
Frame: Frame_0, Subgraph G6, construction finished.
Frame: Frame_0, Subgraph G7, construction finished.
Frame: Frame_0, Subgraph G8, construction finished.
Frame: Frame_0, Subgraph G9, construction finished.
Frame: Frame_0, Subgraph G10, construction finished.
Frame: Frame_0, Subgraph G11, construction finished.
Frame: Frame_0, Subgraph G12, construction finished.
Frame: Frame_0, Subgraph G13, construction finished.
Frame: Frame_0, Subgraph G14, construction finished.
Frame: Frame_0, Subgraph G15, construction finished.
Frame: Frame_0, Subgraph G16, construction finished.
Frame: Frame_0, Subgraph G17, construction finished.
Frame: Frame_0, Subgraph G18, construction finished.
Fra

In [5]:
# import pickle

# # 保存 G 到文件
# with open('Graph_data_huge.pkl', 'wb') as file:
#     pickle.dump(G, file)
print(satellite['LEOS312'])

{'cameras': [], 'connections': {'LEOS311': {'Time': ['25 Jun 2024 02:00:50.000', '25 Jun 2024 02:01:00.000', '25 Jun 2024 02:01:10.000', '25 Jun 2024 02:01:20.000', '25 Jun 2024 02:01:30.000', '25 Jun 2024 02:01:40.000', '25 Jun 2024 02:01:50.000', '25 Jun 2024 02:02:00.000', '25 Jun 2024 02:02:10.000', '25 Jun 2024 02:02:20.000', '25 Jun 2024 02:02:30.000', '25 Jun 2024 02:02:40.000', '25 Jun 2024 02:02:50.000', '25 Jun 2024 02:03:00.000', '25 Jun 2024 02:03:10.000', '25 Jun 2024 02:03:20.000', '25 Jun 2024 02:03:30.000', '25 Jun 2024 02:03:40.000', '25 Jun 2024 02:03:50.000', '25 Jun 2024 02:04:00.000', '25 Jun 2024 02:04:10.000', '25 Jun 2024 02:04:20.000', '25 Jun 2024 02:04:30.000', '25 Jun 2024 02:04:40.000', '25 Jun 2024 02:04:50.000', '25 Jun 2024 02:05:00.000'], 'Range': [1479.697401, 1479.697401, 1479.697401, 1479.697401, 1479.697401, 1479.697401, 1479.697401, 1479.697401, 1479.697401, 1479.697401, 1479.697401, 1479.697401, 1479.697401, 1479.697401, 1479.697401, 1479.697401, 

In [6]:
for subgraph_key in list(G['Frame_0'].keys())[:1]:  # 仅打印前5个子图
    print(f'Subgraph {subgraph_key}:')
    for link, details in G['Frame_0'][subgraph_key].items():
        print(f'{link}: {details}')
    print()

Subgraph G0:
('LEOS101', 'LEOS102'): {'energy': 0.04302308936763554, 'rate': 8.999818601852295, 'outage': 0.014717129109001614}
('LEOS101', 'LEOS130'): {'energy': 0.04301881402829843, 'rate': 9.000713030937904, 'outage': 0.014715930505771267}
('LEOS101', 'LEOS201'): {'energy': 0.15065140601170493, 'rate': 2.5701718307887305, 'outage': 0.04090872305665566}
('LEOS101', 'LEOS301'): {'energy': 0.5705885407659472, 'rate': 0.6785975748482962, 'outage': 0.13315199188797708}
('LEOS101', 'LEOS801'): {'energy': 0.34896144944574886, 'rate': 1.109578151440467, 'outage': 0.08502150262311874}
('LEOS102', 'LEOS101'): {'energy': 0.04301881402829843, 'rate': 9.000713030937904, 'outage': 0.014715930505771267}
('LEOS102', 'LEOS103'): {'energy': 0.04302308936763554, 'rate': 8.999818601852295, 'outage': 0.014717129109001614}
('LEOS102', 'LEOS201'): {'energy': 0.14022337672924262, 'rate': 2.7613084853008822, 'outage': 0.03848614903390921}
('LEOS102', 'LEOS301'): {'energy': 0.5052195515364816, 'rate': 0.7663

In [7]:
import heapq
from collections import defaultdict, deque

class DirectedGraph:
    def __init__(self):
        self.graph = defaultdict(dict)
    
    def add_edge(self, u, v, weight):
        self.graph[u][v] = weight
    
    def edges(self):
        edge_list = []
        for u in self.graph:
            for v, weight in self.graph[u].items():
                edge_list.append((weight, u, v))
        return edge_list

def dijkstra(graph, start):
    pq = [(0, start)]
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    predecessors = {node: None for node in graph}
    visited = set()
    
    while pq:
        current_distance, current_node = heapq.heappop(pq)
        if current_node in visited:
            continue
        visited.add(current_node)
        
        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                predecessors[neighbor] = current_node
                heapq.heappush(pq, (distance, neighbor))
    
    return distances, predecessors

def reverse_graph(G):
    reversed_G = {}
    for src in G.keys():
        for dst in G[src].keys():
            if dst not in reversed_G:
                reversed_G[dst] = {}
            reversed_G[dst][src] = G[src][dst]
    return reversed_G

def graph_to_msa_input(G, root):
    V = set(G.keys())
    E = set()
    w = {}

    for src in G:
        for dst in G[src]:
            E.add((src, dst))
            w[(src, dst)] = G[src][dst]

    return V, E, root, w

def msa(V, E, r, w):
    for (u,v) in E.copy():
        if v == r:
            E.remove((u,v))
            w.pop((u,v))

    pi = dict()
    for v in V:
        edges = [edge[0] for edge in E if edge[1] == v]
        if not len(edges):
            continue
        costs = [w[(u,v)] for u in edges]
        pi[v] = edges[costs.index(min(costs))]
    
    cycle_vertex = None
    for v in V:
        if cycle_vertex is not None:
            break
        visited = set()
        next_v = pi.get(v)
        while next_v:
            if next_v in visited:
                cycle_vertex = next_v
                break
            visited.add(next_v)
            next_v = pi.get(next_v)
    
    if cycle_vertex is None:
        return set([(pi[v],v) for v in pi.keys()])
    
    C = set()
    C.add(cycle_vertex)
    next_v = pi.get(cycle_vertex)
    while next_v != cycle_vertex:
        C.add(next_v)
        next_v = pi.get(next_v)
    
    v_c = -abs(hash(cycle_vertex))
    V_prime = set([v for v in V if v not in C] + [v_c])
    E_prime = set()
    w_prime = dict()
    correspondance = dict()
    for (u,v) in E:
        if u not in C and v in C:
            e = (u,v_c)
            if e in E_prime:
                if w_prime[e] < w[(u,v)] - w[(pi[v],v)]:
                    continue
            w_prime[e] = w[(u,v)] - w[(pi[v],v)]
            correspondance[e] = (u,v)
            E_prime.add(e)
        elif u in C and v not in C:
            e = (v_c,v)
            if e in E_prime:
                old_u = correspondance[e][0]
                if w[(old_u,v)] < w[(u,v)]:
                    continue
            E_prime.add(e)
            w_prime[e] = w[(u,v)]
            correspondance[e] = (u,v)
        elif u not in C and v not in C:
            e = (u,v)
            E_prime.add(e)
            w_prime[e] = w[(u,v)]
            correspondance[e] = (u,v)
    
    tree = msa(V_prime, E_prime, r, w_prime)
    
    cycle_edge = None
    for (u,v) in tree:
        if v == v_c:
            old_v = correspondance[(u,v_c)][1]
            cycle_edge = (pi[old_v],old_v)
            break
    
    ret = set([correspondance[(u,v)] for (u,v) in tree])
    for v in C:
        u = pi[v]
        ret.add((u,v))
        
    ret.remove(cycle_edge)
    
    return ret
    


def build_steiner_tree(subgraph, root, terminals):
    steiner_tree = DirectedGraph()
    total_cost = 0
    rho = 0.1
    # Convert subgraph format to graph format for Dijkstra
    graph = defaultdict(dict)
    all_nodes = set()
    for (u, v), data in subgraph.items():
        graph[u][v] = rho * data['energy'] + (1-rho) * 1 / (1 - math.log10(data['outage']))
        all_nodes.add(u)
        all_nodes.add(v)
    
    # Ensure all terminals are in the graph
    for terminal in terminals:
        all_nodes.add(terminal)
    
    for node in all_nodes:
        if node not in graph:
            graph[node] = {}
    
    # Reverse the graph for Dijkstra
    reversed_graph = reverse_graph(graph)

    # Run Dijkstra from the root in the reversed graph
    distances_to_root, predecessors_to_root = dijkstra(reversed_graph, root)
    
    # Debug: Print distances and predecessors
#     print("Distances to root:", distances_to_root)
#     print("Predecessors to root:", predecessors_to_root)

    # Collect all paths from terminals to the root
    all_edges = set()
    for terminal in terminals:
        current = terminal
        while current != root:
            prev = predecessors_to_root.get(current)
            if prev is None:
                print(f"No path from {current} to root {root}")
                break
            all_edges.add((distances_to_root[current] - distances_to_root[prev], current, prev))
            current = prev

    # Debug: Print all collected edges
#     print("Collected edges for Chu-Liu/Edmonds algorithm:", all_edges)

    # Create subgraph for Chu-Liu/Edmonds algorithm
    G = defaultdict(dict)
    for weight, u, v in all_edges:
        G[u][v] = weight
        
#     print("Number of edges in G:", sum(len(v) for v in G.values()))

    # Add original edges to G
    for u in G:
        for v in graph[u]:
            if u in G and v in G:  # Ensure no overwriting
                G[u][v] = graph[u][v]

    # Print the number of edges in G
#     print("Number of edges in G:", sum(len(v) for v in G.values()))

    # Debug: Print the subgraph G
#     print("Subgraph G for Chu-Liu/Edmonds algorithm:", G)
    
    # Run Chu-Liu/Edmonds algorithm to find the minimum spanning tree
    V, E, r, w = graph_to_msa_input(reverse_graph(G), root)
    result = msa(V, E, r, w)
    
    # Debug: Print the result of the Chu-Liu/Edmonds algorithm
#     print("Result from Chu-Liu/Edmonds algorithm:", result)

    # Reverse the result edges to get the original direction
    final_result = {(v, u) for u, v in result}
    
    # Debug: Print the final result edges
#     print("Final result edges:", final_result)
    
    for u, v in final_result:
        if u in graph and v in graph[u]:  # Ensure the edge exists in the original graph
            steiner_tree.add_edge(u, v, graph[u][v])
            total_cost += graph[u][v]
        else:
            print(f"Edge ({u}, {v}) not found in the original graph.")
            
    # Ensure all terminals are connected
    for terminal in terminals:
        if terminal not in steiner_tree.graph:
            # Recompute path to the root and add to the tree
            current = terminal
#             print(current)
#             while current != root:
#                 prev = predecessors_to_root.get(current)
#                 if prev is None:
#                     break
#                 if prev in graph and current in graph[prev]:
#                     steiner_tree.add_edge(prev, current, graph[prev][current])
#                     total_cost += graph[prev][current]
#                 current = prev
    
    def remove_non_terminal_leaves_and_recompute_cost(tree, terminals):
        to_remove = set()
        cost = 0
        outage_p = 0
        in_degree = defaultdict(int)

        # 计算每个节点的入度
        for u in tree.graph:
            for v in tree.graph[u]:
                in_degree[v] += 1

        # 删除没有入边且不是终端节点的节点
        queue = deque([node for node in tree.graph if in_degree[node] == 0 and node not in terminals])
#         print(queue)
        count = 0
        while queue:
            count += 1
            node = queue.popleft()
            for parent in list(tree.graph.keys()):
                if node in tree.graph[parent]:
                    del tree.graph[parent][node]
                    if len(tree.graph[parent]) == 0 and parent not in terminals:
                        queue.append(parent)
            del tree.graph[node]

        graph_mod = tree
        for u in graph_mod.graph:
            for v in graph_mod.graph[u]:
                cost += graph_mod.graph[u][v]
#                 outage *= 
        if count != 0:
            tree, cost = remove_non_terminal_leaves_and_recompute_cost(graph_mod, terminals)   
        return tree, cost

    tree, cost = remove_non_terminal_leaves_and_recompute_cost(steiner_tree, terminals)
    total_cost = 0
    success = 1
    cnt = 0
    for u in tree.graph:
        for v in tree.graph[u]:
            if v in tree.graph[u]:
                result = random.choices([0, 1], weights=[1-subgraph[(u,v)]['outage'], subgraph[(u,v)]['outage']])[0]
                if result == 1 and rho != 1:
                    total_cost += 2 * subgraph[(u,v)]['energy']
                else:
                    total_cost += subgraph[(u,v)]['energy']
                success = success * (1 - subgraph[(u,v)]['outage'])
                cnt += 1
#             print(outage)
#     tree = steiner_tree
#     cost = total_cost
    return tree, total_cost, 1 - (success**(1/cnt))

def print_graph(graph):
    for node in graph.graph:
        for neighbor, weight in graph.graph[node].items():
            print(f"{node} -> {neighbor} [weight={weight}]")
            
def print_tree(steiner_tree, root):
    reversed_graph = defaultdict(list)
    for u in steiner_tree.graph:
        for v in steiner_tree.graph[u]:
            reversed_graph[v].append((u, steiner_tree.graph[u][v]))
    
    def dfs(node, indent="", last=True):
        connector = "└── " if last else "├── "
        branch = "    " if last else "│   "
        print(indent + connector + node)
        indent += branch
        children = reversed_graph.get(node, [])
        for i, (child, weight) in enumerate(children):
            is_last = (i == len(children) - 1)
            dfs(child, indent, is_last)

    print(f"Tree structure starting from root {root}:")
    dfs(root)
    
random.seed(2024)
i = 0
energy_cost_ours = []
outage_ours = []
for timeframe_key in list(G.keys())[0:1]:
    Frame_data = G[timeframe_key]
    terminals = terminal[i]
    root = root_list[i]
    i += 1
    for subgraph_key in list(Frame_data.keys())[0:25]:  # 仅打印前5个子图
        subgraph = Frame_data[subgraph_key]
        steiner_tree, total_cost,total_outage = build_steiner_tree(subgraph, root, terminals)
        energy_cost_ours.append(total_cost)
        outage_ours.append(total_outage)
        print(f'Frame {timeframe_key}, Subgraph {subgraph_key}: Total cost: {total_cost}, Total outage: {total_outage}')

L_P_GEO = pointing_loss(theta, theta_3dB)
Gain_GEO = max_gain
path_loss_GEO = PL(fc, 36992.141944)
eff_GEO = 0.6
power_to_GEO = satellite[root]['transmit_power']
trans_gain_GEO = satellite[root]['transmit_gain']
recei_gain_GEO = 10 ** (90 / 10)
recei_power_GEO = receiver_power(power_to_GEO, eff_GEO, trans_gain_GEO, recei_gain_GEO, path_loss_GEO, L_P_GEO)
achievable_rate_GEO,_ = rate(Bandwidth, recei_power_GEO, P_N)
gamma_GEO = 10**(-120/10) / (recei_power_GEO / L_P_GEO)
print(gamma_GEO)
outage_GEO, _ = quad(pointing_loss_pdf, 0, gamma, args=(G0, sigma_p))
print(achievable_rate_GEO/10**9,outage_GEO)
# print(achievable_rate_GEO/10**9)
U_GEO = 1
energy_GEO, time_GEO = weight(achievable_rate_GEO, s,U_GEO, power_to_GEO)
total_energy_cost_ours = sum(energy_cost_ours)
average_outage_per_edge = sum(outage_ours) / 25
total_energy_cost_GEO_ours = total_energy_cost_ours + energy_GEO
print("Energy_Consumption:",total_energy_cost_ours)
print("Outage:",average_outage_per_edge)
print("Energy_Consumption_GEO:",total_energy_cost_GEO_ours)
# 示例用法
# subgraph = G['G0']

# print(energy_cost)
# print(terminals)
# steiner_tree, total_cost = build_steiner_tree(subgraph, root, terminals)
# print("Steiner Tree Edges:", steiner_tree.edges())
# print("Total Cost:", total_cost)
# print_graph(steiner_tree)
# print_tree(steiner_tree,root)
# print_tree(steiner_tree,root)

Frame Frame_0, Subgraph G0: Total cost: 0.5675318563628257, Total outage: 0.01376220341089085
Frame Frame_0, Subgraph G1: Total cost: 0.6172360450857335, Total outage: 0.013902266434157928
Frame Frame_0, Subgraph G2: Total cost: 0.5814016150041611, Total outage: 0.014051815286050595
Frame Frame_0, Subgraph G3: Total cost: 0.6800757619979803, Total outage: 0.014210561030298385
Frame Frame_0, Subgraph G4: Total cost: 0.639392753403873, Total outage: 0.014378211023654996
Frame Frame_0, Subgraph G5: Total cost: 0.6038868371822922, Total outage: 0.014537270171153915
Frame Frame_0, Subgraph G6: Total cost: 0.6099521797397541, Total outage: 0.014661570343338148
Frame Frame_0, Subgraph G7: Total cost: 0.6163963435360933, Total outage: 0.014792381718738978
Frame Frame_0, Subgraph G8: Total cost: 0.6232164506361875, Total outage: 0.014929476754495075
Frame Frame_0, Subgraph G9: Total cost: 0.63040943579003, Total outage: 0.015072642961422411
Frame Frame_0, Subgraph G10: Total cost: 0.63797207285

In [8]:
import heapq
from collections import defaultdict

class DirectedGraph:
    def __init__(self):
        self.graph = defaultdict(dict)
    
    def add_edge(self, u, v, weight):
        self.graph[u][v] = weight
    
    def edges(self):
        edge_list = []
        for u in self.graph:
            for v, weight in self.graph[u].items():
                edge_list.append((weight, u, v))
        return edge_list

def dijkstra(graph, start):
    pq = [(0, start)]
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    predecessors = {node: None for node in graph}
    visited = set()
    
    while pq:
        current_distance, current_node = heapq.heappop(pq)
        if current_node in visited:
            continue
        visited.add(current_node)
        
        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                predecessors[neighbor] = current_node
                heapq.heappush(pq, (distance, neighbor))
    
    return distances, predecessors

def find(parent, i):
    if parent[i] == i:
        return i
    else:
        parent[i] = find(parent, parent[i])
        return parent[i]

def union(parent, rank, x, y):
    root_x = find(parent, x)
    root_y = find(parent, y)
    
    if root_x != root_y:
        if rank[root_x] > rank[root_y]:
            parent[root_y] = root_x
        elif rank[root_x] < rank[root_y]:
            parent[root_x] = root_y
        else:
            parent[root_y] = root_x
            rank[root_x] += 1

        
def build_steiner_tree(subgraph, root, terminals):
    steiner_tree = DirectedGraph()
    total_cost = 0
    
    rho = 0.1
    # Convert subgraph format to graph format for Dijkstra
    graph = defaultdict(dict)
    all_nodes = set()
    for (u, v), data in subgraph.items():
        graph[u][v] = rho * data['energy'] + (1-rho) * 1 / (1 - math.log10(data['outage']))
        all_nodes.add(u)
        all_nodes.add(v)
    
    # Ensure all terminals are in the graph
    for terminal in terminals:
        all_nodes.add(terminal)
    
    for node in all_nodes:
        if node not in graph:
            graph[node] = {}
    
    # Reverse the graph for Dijkstra
    reversed_graph = reverse_graph(graph)

    # Run Dijkstra from the root in the reversed graph
    distances_to_root, predecessors_to_root = dijkstra(reversed_graph, root)
    
    # Debug: Print distances and predecessors
#     print("Distances to root:", distances_to_root)
#     print("Predecessors to root:", predecessors_to_root)

    # Collect all paths from terminals to the root
    all_edges = set()
    for terminal in terminals:
        current = terminal
        while current != root:
            prev = predecessors_to_root.get(current)
            if prev is None:
                print(f"No path from {current} to root {root}")
                break
            all_edges.add((distances_to_root[current] - distances_to_root[prev], current, prev))
            current = prev

    # Debug: Print all collected edges
#     print("Collected edges for Chu-Liu/Edmonds algorithm:", all_edges)

    all_edges = sorted(all_edges)
    parent = {}
    rank = {}
    for node in graph:
        parent[node] = node
        rank[node] = 0
    
    for weight, u, v in all_edges:
        if find(parent, u) != find(parent, v):
            steiner_tree.add_edge(u, v, weight)
#             total_cost += weight
            union(parent, rank, u, v)
    
    total_cost = 0
    success = 1
    cnt = 0
    for u in steiner_tree.graph:
        for v in steiner_tree.graph[u]:
            if v in steiner_tree.graph[u]:
                result = random.choices([0, 1], weights=[1-subgraph[(u,v)]['outage'], subgraph[(u,v)]['outage']])[0]
                if result == 1 and rho != 1:
                    total_cost += 2 * subgraph[(u,v)]['energy']
                else:
                    total_cost += subgraph[(u,v)]['energy']
                success = success * (1 - subgraph[(u,v)]['outage'])
                cnt += 1
    # Debug: Print the subgraph G
#     print("Subgraph G for Chu-Liu/Edmonds algorithm:", G)
    
    # Run Chu-Liu/Edmonds algorithm to find the minimum spanning tree
    
#     for u, v in final_result:
#         if u in graph and v in graph[u]:  # Ensure the edge exists in the original graph
#             steiner_tree.add_edge(u, v, graph[u][v])
#             total_cost += graph[u][v]
#         else:
#             print(f"Edge ({u}, {v}) not found in the original graph.")
            
    # Ensure all terminals are connected
    for terminal in terminals:
        if terminal not in steiner_tree.graph:
            # Recompute path to the root and add to the tree
            current = terminal
#             print(current)
#             while current != root:
#                 prev = predecessors_to_root.get(current)
#                 if prev is None:
#                     break
#                 if prev in graph and current in graph[prev]:
#                     steiner_tree.add_edge(prev, current, graph[prev][current])
#                     total_cost += graph[prev][current]
#                 current = prev
    
    return steiner_tree, total_cost, 1 - (success ** (1/cnt))

def print_graph(graph):
    for node in graph.graph:
        for neighbor, weight in graph.graph[node].items():
            print(f"{node} -> {neighbor} [weight={weight}]")
            
def print_tree(steiner_tree, root):
    reversed_graph = defaultdict(list)
    for u in steiner_tree.graph:
        for v in steiner_tree.graph[u]:
            reversed_graph[v].append((u, steiner_tree.graph[u][v]))
    
    def dfs(node, indent="", last=True):
        connector = "└── " if last else "├── "
        branch = "    " if last else "│   "
        print(indent + connector + node)
        indent += branch
        children = reversed_graph.get(node, [])
        for i, (child, weight) in enumerate(children):
            is_last = (i == len(children) - 1)
            dfs(child, indent, is_last)

    print(f"Tree structure starting from root {root}:")
    dfs(root)

# 示例用法


random.seed(2024)
i = 0
energy_cost_merge = []
outage_merge = []
for timeframe_key in list(G.keys())[0:1]:
    Frame_data = G[timeframe_key]
    terminals = terminal[i]
    root = root_list[i]
    i += 1
    for subgraph_key in list(Frame_data.keys())[0:25]:  # 仅打印前5个子图
        subgraph = Frame_data[subgraph_key]
        steiner_tree, total_cost, total_outage = build_steiner_tree(subgraph, root, terminals)
        energy_cost_merge.append(total_cost)
        outage_merge.append(total_outage)
        print(f'Frame {timeframe_key}, Subgraph {subgraph_key}: Total cost: {total_cost}, Total outage: {total_outage}')
    
total_energy_cost_merge = sum(energy_cost_merge) 
average_outage_per_edge = sum(outage_merge) / 25
total_energy_cost_GEO_merge = total_energy_cost_merge + energy_GEO
print("Energy_Consumption:",total_energy_cost_merge)
print("Outage:",average_outage_per_edge)
print("Energy_Consumption_GEO:",total_energy_cost_GEO_merge)


# root = 'LEOS310'
# terminals = [sat for sat in satellite if satellite[sat]['cameras']]
# print(terminals)
# steiner_tree, total_cost = build_steiner_tree(subgraph, root, terminals)
# print("Steiner Tree Edges:", steiner_tree.edges())
# print("Total Cost:", total_cost)
# print_graph(steiner_tree)
# print_tree(steiner_tree,root)

Frame Frame_0, Subgraph G0: Total cost: 0.725337651698555, Total outage: 0.015600497543271752
Frame Frame_0, Subgraph G1: Total cost: 0.759830244357029, Total outage: 0.015931612761487135
Frame Frame_0, Subgraph G2: Total cost: 0.7629229223021066, Total outage: 0.016276074752968994
Frame Frame_0, Subgraph G3: Total cost: 0.8260093921685009, Total outage: 0.01663349818686144
Frame Frame_0, Subgraph G4: Total cost: 0.8290831823724931, Total outage: 0.017003499440044312
Frame Frame_0, Subgraph G5: Total cost: 0.8256095630775966, Total outage: 0.017385699338289373
Frame Frame_0, Subgraph G6: Total cost: 0.848150324630161, Total outage: 0.01777972761325841
Frame Frame_0, Subgraph G7: Total cost: 0.8714971425557103, Total outage: 0.018185227076600308
Frame Frame_0, Subgraph G8: Total cost: 0.9109383351620142, Total outage: 0.018601857227628193
Frame Frame_0, Subgraph G9: Total cost: 0.9205667337505495, Total outage: 0.019029294897762616
Frame Frame_0, Subgraph G10: Total cost: 0.946267490948

In [9]:
import heapq
import random
import re
from collections import defaultdict, deque

random.seed(2024)

class DirectedGraph:
    def __init__(self):
        self.graph = defaultdict(dict)
    
    def add_edge(self, u, v, weight):
        self.graph[u][v] = weight
    
    def edges(self):
        edge_list = []
        for u in self.graph:
            for v, weight in self.graph[u].items():
                edge_list.append((weight, u, v))
        return edge_list

def dijkstra(graph, start):
    pq = [(0, start)]
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    predecessors = {node: None for node in graph}
    visited = set()
    
    while pq:
        current_distance, current_node = heapq.heappop(pq)
        if current_node in visited:
            continue
        visited.add(current_node)
        
        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                predecessors[neighbor] = current_node
                heapq.heappush(pq, (distance, neighbor))
    
    return distances, predecessors

def find_min_arc(terminals, graph, orbit):
    terminal_indices = sorted([int(re.search(r'(\d+)', t).group()) % 100 for t in terminals])
    start = terminal_indices[0]
    end = terminal_indices[-1]

    # Determine if the arc should wrap around the end of the orbit
    direct_arc = [f'LEOS{orbit*100 + i}' for i in range(start, end + 1)]
    wrap_arc = [f'LEOS{orbit*100 + i}' for i in range(start, 21)] + [f'LEOS{orbit*100 + i}' for i in range(1, end + 1)]
    
    if len(direct_arc) <= 20:
        min_arc = direct_arc
    else:
        min_arc = wrap_arc
    
    return min_arc

def build_steiner_tree(subgraph, terminals):
    steiner_tree = DirectedGraph()
    total_cost = 0
    rho = 0.1
    # Convert subgraph format to graph format for Dijkstra
    graph = defaultdict(dict)
    all_nodes = set()
    for (u, v), data in subgraph.items():
        graph[u][v] = rho * data['energy'] + (1-rho) * 1 / (1 - math.log10(data['outage']))
        all_nodes.add(u)
        all_nodes.add(v)
    
    # Ensure all terminals are in the graph
    for terminal in terminals:
        all_nodes.add(terminal)
    
    for node in all_nodes:
        if node not in graph:
            graph[node] = {}
    
    orbits = defaultdict(list)
    for terminal in terminals:
        orbit = int(re.search(r'(\d+)', terminal).group()) // 100
        orbits[orbit].append(terminal)
    
    for orbit, orbit_terminals in orbits.items():
        min_arc = find_min_arc(orbit_terminals, graph, orbit)
#         print(min_arc)
        
        if min_arc:
            min_arc_graph = defaultdict(dict)
            for node in min_arc:
                for neighbor in graph[node]:
                    if neighbor in min_arc:
                        min_arc_graph[node][neighbor] = graph[node][neighbor]
                # Ensure all nodes in min_arc have entries in min_arc_graph, even if they have no edges
                if node not in min_arc_graph:
                    min_arc_graph[node] = {}

            root = random.choice(min_arc)
            for terminal in orbit_terminals:
                if terminal != root:
                    distances, predecessors = dijkstra(min_arc_graph, root)
                    current = terminal
                    while current != root:
                        prev = predecessors.get(current)
                        if prev is None:
                            break
                        steiner_tree.add_edge(prev, current, graph[prev][current])
#                         total_cost += graph[prev][current]
                        current = prev
    total_cost = 0
    success = 1
    cnt = 0
    for u in steiner_tree.graph:
        for v in steiner_tree.graph[u]:
            if v in steiner_tree.graph[u]:
                result = random.choices([0, 1], weights=[1-subgraph[(u,v)]['outage'], subgraph[(u,v)]['outage']])[0]
                if result == 1 and rho != 1:
                    total_cost += 2 * subgraph[(u,v)]['energy']
                else:
                    total_cost += subgraph[(u,v)]['energy']
                success = success * (1 - subgraph[(u,v)]['outage'])
                cnt += 1
    return steiner_tree, total_cost, 1 - (success ** (1/cnt))

def print_graph(graph):
    for node in graph.graph:
        for neighbor, weight in graph.graph[node].items():
            print(f"{node} -> {neighbor} [weight={weight}]")
            
def print_tree(steiner_tree, root):
    reversed_graph = defaultdict(list)
    for u in steiner_tree.graph:
        for v in steiner_tree.graph[u]:
            reversed_graph[v].append((u, steiner_tree.graph[u][v]))
    
    def dfs(node, indent="", last=True):
        connector = "└── " if last else "├── "
        branch = "    " if last else "│   "
        print(indent + connector + node)
        indent += branch
        children = reversed_graph.get(node, [])
        for i, (child, weight) in enumerate(children):
            is_last = (i == len(children) - 1)
            dfs(child, indent, is_last)

    print(f"Tree structure starting from root {root}:")
    dfs(root)

# 示例用法
energy_cost_greedy = []
outage_greedy = []
random.seed(2024)
i = 0
for timeframe_key in list(G.keys())[0:1]:
    Frame_data = G[timeframe_key]
    terminals = terminal[i]
    root = root_list[i]
    i += 1
    for subgraph_key in list(Frame_data.keys())[0:25]:  # 仅打印前5个子图
        subgraph = Frame_data[subgraph_key]
        steiner_tree, total_cost, total_outage = build_steiner_tree(subgraph, terminals)
        energy_cost_greedy.append(total_cost)
        outage_greedy.append(total_outage)
        print(f'Frame {timeframe_key}, Subgraph {subgraph_key}: Total cost: {total_cost}, Total outage: {total_outage}')
    
total_energy_cost_greedy = sum(energy_cost_greedy)
average_outage_per_edge = sum(outage_greedy) / 25
total_energy_cost_GEO_greedy = total_energy_cost_greedy + energy_GEO * 4
print("Energy_Consumption:",total_energy_cost_greedy)
print("Outage:",average_outage_per_edge)
print("Energy_Consumption_GEO:",total_energy_cost_GEO_greedy)





# print_graph(steiner_tree)

Frame Frame_0, Subgraph G0: Total cost: 0.4732240556686312, Total outage: 0.014716366361660072
Frame Frame_0, Subgraph G1: Total cost: 0.4732368816866425, Total outage: 0.014716693253450153
Frame Frame_0, Subgraph G2: Total cost: 0.4732283310079683, Total outage: 0.014716475325602163
Frame Frame_0, Subgraph G3: Total cost: 0.5162300436789182, Total outage: 0.014716039469761522
Frame Frame_0, Subgraph G4: Total cost: 0.4732283310079683, Total outage: 0.014716475325602163
Frame Frame_0, Subgraph G5: Total cost: 0.47324115702597963, Total outage: 0.014716802217356051
Frame Frame_0, Subgraph G6: Total cost: 0.4732283310079683, Total outage: 0.014716475325602163
Frame Frame_0, Subgraph G7: Total cost: 0.47324970770465385, Total outage: 0.014717020145131765
Frame Frame_0, Subgraph G8: Total cost: 0.47321122965061985, Total outage: 0.014716039469761522
Frame Frame_0, Subgraph G9: Total cost: 0.4732368816866425, Total outage: 0.014716693253450153
Frame Frame_0, Subgraph G10: Total cost: 0.4732